In [1]:
import sys
sys.path.append("/mnt/home/tf_transformers2/")

In [2]:
from transformers import TFAlbertModel
from tf_transformers.models import AlbertEncoder

import tensorflow as tf
import json

from tf_transformers.utils import convert_albert_hf_to_tf_transformers

/usr/lib/python3/dist-packages/Crypto/Random/Fortuna/FortunaGenerator.py:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if sys.version_info[0] is 2 and  sys.version_info[1] is 1:
/usr/lib/python3/dist-packages/Crypto/Random/Fortuna/FortunaGenerator.py:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if sys.version_info[0] is 2 and  sys.version_info[1] is 1:


In [3]:
# Load HF model

# Always do this
tf.keras.backend.clear_session()

model_hf_location = '/mnt/home/PRE_MODELS/HuggingFace_models/albert-base-v2/'
model_hf = TFAlbertModel.from_pretrained(model_hf_location)

All model checkpoint weights were used when initializing TFAlbertModel.

All the weights of TFAlbertModel were initialized from the model checkpoint at /mnt/home/PRE_MODELS/HuggingFace_models/albert-base-v2/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


In [4]:
# Load tf_transformers model
# Most config we will be providing

# Default configs for the model
config_location = '/mnt/home/tf_transformers2/tf_transformers/model_directory/albert_base_v2/albert_config.json'
config = json.load(open(config_location))

# Always do this
tf.keras.backend.clear_session()

# tf_transformers Layer (an extension of Keras Layer)
# This is not Keras model, but extension of keras Layer


model_layer = AlbertEncoder(config=config,
                      name='albert',
                      mask_mode=config['mask_mode'],
                      is_training=False, 
                      use_dropout=False,
                      )
# model_dir = None, because we have not initialized the model with proper variable values
model_tf_transformers = model_layer.get_and_load_model(model_dir=None)

INFO:absl:We are overwriding `is_training` is False to `is_training` to True with `use_dropout` is False, no effects on your inference pipeline
INFO:absl:Inputs -->
INFO:absl:input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:input_type_ids ---> Tensor("input_type_ids:0", shape=(None, None), dtype=int32)
INFO:absl:Initialized Variables
INFO:absl:Inputs -->
INFO:absl:input_ids ---> Tensor("input_ids_1:0", shape=(None, None), dtype=int32)
INFO:absl:input_mask ---> Tensor("input_mask_1:0", shape=(None, None), dtype=int32)
INFO:absl:input_type_ids ---> Tensor("input_type_ids_1:0", shape=(None, None), dtype=int32)


In [5]:
convert_albert_hf_to_tf_transformers(model_hf, model_tf_transformers, config)

INFO:absl:Deleteing huggingface model for saving memory
INFO:absl:Done assigning variables weights. Total 25


In [6]:
# Please have a look at tf_transformers/extra/*.py for reference values

input_ids  = tf.constant([[1, 9, 10, 11, 23], 
                         [1, 22, 234, 432, 2349]])
input_mask = tf.ones_like(input_ids)
input_type_ids = tf.ones_like(input_ids)

inputs = {'input_ids': input_ids, 
          'input_mask': input_mask, 
          'input_type_ids': input_type_ids}

results_tf_transformers   = model_tf_transformers(inputs)
for k, r in results_tf_transformers.items():
    if isinstance(r, list):
        continue
    print(k, '-->', tf.reduce_sum(r), '-->', r.shape)

cls_output --> tf.Tensor(12.337944, shape=(), dtype=float32) --> (2, 768)
token_embeddings --> tf.Tensor(-193.53145, shape=(), dtype=float32) --> (2, 5, 768)


In [7]:
# Huggingface Model
input_ids  = tf.constant([[1, 9, 10, 11, 23], 
                         [1, 22, 234, 432, 2349]])
input_mask = tf.ones_like(input_ids)
input_type_ids = tf.ones_like(input_ids)

results_hf = model_hf([input_ids, input_mask, input_type_ids])
for k in results_hf:
    print(tf.reduce_sum(k), '-->', k.shape)

tf.Tensor(-193.53175, shape=(), dtype=float32) --> (2, 5, 768)
tf.Tensor(12.337847, shape=(), dtype=float32) --> (2, 768)
